# Load small dandiset

author: laquitainesteeve@gmail.com

purpose: create small demo dataset (neuropixels evoked 20 KHz)

# Setup 

Activate dandi virtual environment (envs/dandi.yml)

```bash
python -m ipykernel install --user --name dandi --display-name "dandi"
```